# Answer of Exercise 2:
 
 Build a neural network with a minimun of 2 layers in order to do classification for image dataset.

In [6]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import time
import pdb
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline

torch.manual_seed(1)    # reproducible

In [7]:
def get_train_valid_loader(data_dir='../data',
                           batch_size=64,
                           augment=False,
                           random_seed = 1,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=9, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    return (train_loader, valid_loader)

trainloader, valloader = get_train_valid_loader()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden1, n_hidden2, n_hidden3, n_hidden4, n_output):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_feature, n_hidden1)
        self.relu = nn.LeakyReLU()
        self.fc2 = nn.Linear(n_hidden1,n_hidden2)
        self.relu2 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(n_hidden2,n_hidden3)
        self.relu3 = nn.LeakyReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(n_hidden3,n_hidden4)
        self.relu4 = nn.LeakyReLU()
        self.dropout3 = nn.Dropout(0.3)
        self.fc5 = nn.Linear(n_hidden4,n_output)

    def forward(self, x):
        x = x.view(x.size(0),-1)
        #print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout1(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout2(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.dropout3(x)
        x = self.fc5(x)
        return x

In [9]:

net = Net(n_feature=3072, n_hidden1=3000, n_hidden2=2000,n_hidden3=1000,n_hidden4=100, n_output=10)     # define the network
print(net)  # net architecture

learning_rate = 1.5e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  # Choose the optimizer you want and tune its hyperparameter
loss_func = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

Net(
  (fc1): Linear(in_features=3072, out_features=3000, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (fc2): Linear(in_features=3000, out_features=2000, bias=True)
  (relu2): LeakyReLU(negative_slope=0.01)
  (dropout1): Dropout(p=0.1)
  (fc3): Linear(in_features=2000, out_features=1000, bias=True)
  (relu3): LeakyReLU(negative_slope=0.01)
  (dropout2): Dropout(p=0.2)
  (fc4): Linear(in_features=1000, out_features=100, bias=True)
  (relu4): LeakyReLU(negative_slope=0.01)
  (dropout3): Dropout(p=0.3)
  (fc5): Linear(in_features=100, out_features=10, bias=True)
)


In [ ]:
#traindataset = utils.TensorDataset(X_train, y_train)
#trainloader = utils.DataLoader(traindataset, batch_size=64, shuffle=True)
def adjust_learning_rate(lr,update_lr, optimizer, epoch):
    lr = lr * (0.8 ** (epoch // update_lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
best_val = 0.0

epochs = 12
update_lr = 4
steps = 0
print_every = 20
for e in range(epochs):
    adjust_learning_rate(learning_rate,update_lr,optimizer,e)
    start = time.time()
    for images, labels in iter(trainloader):
        steps += 1
        #print(images.dtype)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = loss_func(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if steps % print_every == 0:
                stop = time.time() 
                # Test accuracy
                net.eval()
                correct = 0
                total = 0
                with torch.no_grad():
                    for data in valloader:
                        images, labels = data
                        images = images.to(device)
                        labels = labels.to(device)
                        outputs = net(images)
                        _, predicted = torch.max(F.softmax(outputs).data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                    accuracy = 100.0 * correct / total
                    print('Accuracy of the network on the %f val images: \
                    %f %%' % (total,accuracy))
                    if (accuracy > best_val):
                        best_val = accuracy
                        torch.save(net.state_dict(),'model.ckpt')

                start = time.time()

cpu


/home/alirezamsh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 5000.000000 val images:                     26.480000 %
Accuracy of the network on the 5000.000000 val images:                     30.360000 %
Accuracy of the network on the 5000.000000 val images:                     32.480000 %
Accuracy of the network on the 5000.000000 val images:                     34.020000 %
Accuracy of the network on the 5000.000000 val images:                     35.860000 %
Accuracy of the network on the 5000.000000 val images:                     34.840000 %
Accuracy of the network on the 5000.000000 val images:                     37.300000 %
Accuracy of the network on the 5000.000000 val images:                     39.700000 %
Accuracy of the network on the 5000.000000 val images:                     38.700000 %
Accuracy of the network on the 5000.000000 val images:                     39.500000 %
Accuracy of the network on the 5000.000000 val images:                     41.600000 %
Accuracy of the network on the 5000.000000 

Accuracy of the network on the 5000.000000 val images:                     50.840000 %
Accuracy of the network on the 5000.000000 val images:                     50.500000 %
Accuracy of the network on the 5000.000000 val images:                     51.760000 %
Accuracy of the network on the 5000.000000 val images:                     52.160000 %
Accuracy of the network on the 5000.000000 val images:                     51.760000 %
Accuracy of the network on the 5000.000000 val images:                     51.300000 %
Accuracy of the network on the 5000.000000 val images:                     51.500000 %


After training, the model should be saved to be tested on the test dataset or to be used in a real-life application. To save a model in pytorch:

In [ ]:
torch.save(net.state_dict(), 'model.ckpt')

To load a pretrained model:

In [ ]:
checkpoint = torch.load("model.ckpt")
net.load_state_dict(checkpoint)